In [102]:
# imports
import os
import io
import sys
from openai import OpenAI
import gradio as gr
import subprocess 
from IPython.display import Markdown, display

In [103]:
OLLAMA_BASE_URL = "http://localhost:11434/v1"
openai = OpenAI()
ollama = OpenAI(base_url=OLLAMA_BASE_URL, api_key='ollama')  

In [104]:
models = ["gpt-oss:20b","llama3.2:latest","llama3.1:8b","gemma3:270m",]
clients ={"gpt-oss:20b":ollama, "llama3.2:latest":ollama, "llama3.1:8b":ollama, "gemma3:270m":ollama}

In [105]:
from system_info import retrieve_system_info
system_info = retrieve_system_info()
system_info

{'os': {'system': 'Windows',
  'arch': 'AMD64',
  'release': '11',
  'version': '10.0.26200',
  'kernel': '11',
  'distro': None,
  'wsl': False,
  'rosetta2_translated': False,
  'target_triple': 'mingw32'},
 'package_managers': ['winget'],
 'cpu': {'brand': 'AMD Ryzen 7 5800HS with Radeon Graphics',
  'cores_logical': 16,
  'cores_physical': 8,
  'simd': []},
 'toolchain': {'compilers': {'gcc': 'gcc.exe (MinGW.org GCC-6.3.0-1) 6.3.0',
   'g++': 'g++.exe (MinGW.org GCC-6.3.0-1) 6.3.0',
   'clang': '',
   'msvc_cl': ''},
  'build_tools': {'cmake': '', 'ninja': '', 'make': ''},
  'linkers': {'ld_lld': ''}}}

In [106]:
compile_command = [
                "g++",
                "main.cpp",
                "-o",
                "main.exe"
]

run_command = ["main.exe"]


In [107]:
system_prompt = """
Your task is to convert Python code into high performance C++ code.
Respond only with C++ code. Do not provide any explanation other than occasional comments.
The C++ response needs to produce an identical output in the fastest possible time.
"""

def user_prompt_for(python):
    return f"""
Port this Python code to C++ with the fastest possible implementation that produces identical output in the least time.
The system information is:
{system_info}
Your response will be written to a file called main.cpp and then compiled and executed; the compilation command is:
{compile_command}
Respond only with C++ code.
Python code to port:

```python
{python}
```
"""

In [108]:

def messages_for(python):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(python)}
    ]


In [109]:

def write_output(cpp):
    with open("main.cpp", "w") as f:
        f.write(cpp)

In [110]:

def port(model, python):
    client = clients[model]
    response = client.chat.completions.create(model=model, messages=messages_for(python))
    reply = response.choices[0].message.content
    reply = reply.replace('```cpp','').replace('```','')
    write_output(reply)
    return reply

In [111]:
pi = """
import time

def calculate(iterations, param1, param2):
    result = 1.0
    for i in range(1, iterations+1):
        j = i * param1 - param2
        result -= (1/j)
        j = i * param1 + param2
        result += (1/j)
    return result

start_time = time.time()
result = calculate(200_000_000, 4, 1) * 4
end_time = time.time()

print(f"Result: {result:.12f}")
print(f"Execution Time: {(end_time - start_time):.6f} seconds")
"""

In [112]:
def run_python(code):
    globals_dict = {"__builtins__": __builtins__}

    buffer = io.StringIO()
    old_stdout = sys.stdout
    sys.stdout = buffer

    try:
        exec(code, globals_dict)
        output = buffer.getvalue()
    except Exception as e:
        output = f"Error: {e}"
    finally:
        sys.stdout = old_stdout

    return output

In [113]:
run_python(pi)

'Result: 3.141592656089\nExecution Time: 32.336823 seconds\n'

In [117]:

def compile_and_run():
    try:
        subprocess.run(compile_command, check=True, text=True, capture_output=True)
        print(subprocess.run(run_command, check=True, text=True, capture_output=True).stdout)
        print(subprocess.run(run_command, check=True, text=True, capture_output=True).stdout)
        print(subprocess.run(run_command, check=True, text=True, capture_output=True).stdout)
        print(subprocess.run(run_command, check=True, text=True, capture_output=True).stdout)
    except subprocess.CalledProcessError as e:
        print(f"An error occurred:\n{e.stderr}")

In [115]:
with gr.Blocks() as ui:
    with gr.Row():
        python = gr.Textbox(label="Python code:", lines=28, value=pi)
        cpp = gr.Textbox(label="C++ code:", lines=28)
    with gr.Row():
        model = gr.Dropdown(models, label="Select model", value=models[0])
        convert = gr.Button("Convert code")

    convert.click(port, inputs=[model, python], outputs=[cpp])

ui.launch(inbrowser=True)

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


In [120]:

compile_and_run()

Result: 3.141592656089
Execution Time: 10.419358 seconds

Result: 3.141592656089
Execution Time: 8.394830 seconds

Result: 3.141592656089
Execution Time: 8.392871 seconds

Result: 3.141592656089
Execution Time: 8.350092 seconds

